<a href="https://colab.research.google.com/github/hihohub/umbrella-networks/blob/main/joint_probability_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

JOINT PROBABILITY NETWORK
- using just one folder structure (hierarchical for an umbrella network), the program also makes a general softmax model (levels = 0) from the bottom level folders, then combines the predictions of the general model with the predictions of the top level umbrella model
- their predictions are combined with joint probability multiplication

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
! wget https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py

--2022-04-18 00:06:21--  https://raw.githubusercontent.com/hihohub/umbrella-networks/main/umbrella.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49668 (49K) [text/plain]
Saving to: ‘umbrella.py’

umbrella.py         100%[===================>]  48.50K  --.-KB/s    in 0.01s   

2022-04-18 00:06:22 (4.65 MB/s) - ‘umbrella.py’ saved [49668/49668]



In [3]:
from umbrella import Umbrella_Network,Umbrella_Label,Umbrella_Node,Joint_Probability_Network

In [4]:
import os
import sys
import numpy as np
import math
import random
from PIL import Image, ImageOps, ImageChops
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Flatten, Dense, add, BatchNormalization, Dropout, AveragePooling2D, GlobalAveragePooling2D, SeparableConv2D
from tensorflow.keras.utils import to_categorical
from tensorflow.python.framework.ops import disable_eager_execution
from tensorflow.keras import backend as backend
from keras.callbacks import ReduceLROnPlateau
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import pickle
import copy
import time

In [5]:
images_folder = "/content/drive/My Drive/5390/project/small_hierarchy"
serial_file = "/content/drive/My Drive/Colab Notebooks/small_hierarchy.dat"
models_folder = "/content/drive/My Drive/Colab Notebooks/models"
target_image_size = 30
channels = 3
valid_ratio = 0.1 # validation set
test_ratio = 0.0 # test set
Model = Joint_Probability_Network(images_folder,serial_file,models_folder,target_image_size,channels,valid_ratio,test_ratio)

In [6]:
Model.traverse_training_data()

specialist
small_hierarchy trainX 154 (154,) trainY 154 (154, 3)
   animals trainX 0 (0,) trainY 0 (0,)
      alligator trainX 0 (0,) trainY 0 (0,)
      mammals trainX 0 (0,) trainY 0 (0,)
      salamanders trainX 0 (0,) trainY 0 (0,)
      snake trainX 0 (0,) trainY 0 (0,)
   objects trainX 0 (0,) trainY 0 (0,)
      automobiles trainX 0 (0,) trainY 0 (0,)
      bicycles trainX 0 (0,) trainY 0 (0,)
      boats trainX 0 (0,) trainY 0 (0,)
   plants trainX 0 (0,) trainY 0 (0,)
      cactus trainX 0 (0,) trainY 0 (0,)
      hedges trainX 0 (0,) trainY 0 (0,)
      shrubs trainX 0 (0,) trainY 0 (0,)
generalist
small_hierarchy trainX 154 (154,) trainY 154 (154, 10)
   alligator trainX 0 (0,) trainY 0 (0,)
   mammals trainX 0 (0,) trainY 0 (0,)
   salamanders trainX 0 (0,) trainY 0 (0,)
   snake trainX 0 (0,) trainY 0 (0,)
   automobiles trainX 0 (0,) trainY 0 (0,)
   bicycles trainX 0 (0,) trainY 0 (0,)
   boats trainX 0 (0,) trainY 0 (0,)
   cactus trainX 0 (0,) trainY 0 (0,)
   hedges t

In [7]:
Model.make_models()

making models for specialist
making model for small_hierarchy
making modes for generalist
making model for small_hierarchy


In [8]:
Model.train_models(eps=1,batch=10,aug=False,vrb=True)

training specialist
training model for small_hierarchy
trainX 154
trainY 154
16/16 [==============================] - 10s 6ms/step - loss: 1.1066 - accuracy: 0.3506
training generalist
training model for small_hierarchy
trainX 154
trainY 154
16/16 [==============================] - 0s 3ms/step - loss: 2.3168 - accuracy: 0.0974


In [9]:
logits = Model.predict_validation_set(verbose=True)

validation accuracy
specialist
expected [[1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [1, 0, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 1, 0], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]]
predicted [[0.19321130381936036, 0.525202776225047, 0.28158591995559273], [0.19135936974450213, 0.5201686974818557, 0.2884719327736422], [0.16894348230341089, 0.459235997981954, 0.3718205197146351], [0.20513372060907079, 0.5576112651358318, 0.2372550142550974], [0.18962782992270844, 0.5154618842490207, 0.29491028582827084], [0.21091701746586733, 0.5733318958902461, 0.21575108664388648], [0.25877931379787783, 0.5418757413060804, 0.1993449448960419], [0.2613111187879523, 0.5400248435486681, 0.1986640376633796], [0.20029102887195302, 0.5444474641859959, 0.25526150694205113], [0.18941341833273437, 0.5148790531201831, 0.29570752854708254], [0.23445528436255209, 0.559658031691624, 0.205886683945824], [0.19556712064942133, 0.5316065503053797, 0.27282632904519905], [0.1

In [10]:
for logit in logits:
  print(str(logit))

[0.014229193202229347, 0.02197991029187164, 0.01906388746778022, 0.015935979149022353, 0.04624722859690942, 0.050461922888996154, 0.060399076405381225, 0.01696487882483072, 0.030492739423326677, 0.046115321831546985]
[0.01842750752038892, 0.027823891759056182, 0.025867400471655495, 0.017134914290214145, 0.029018039672195108, 0.04148011228386787, 0.04736991585690183, 0.015430401467592208, 0.03161865042000454, 0.041507445095140344]
[0.011057415105972015, 0.017290385109184094, 0.014595197573969904, 0.01328945744031772, 0.047664004937396504, 0.046639550661361016, 0.05679307467834643, 0.023203336836058453, 0.03944893900574274, 0.0630732088810721]
[0.019926047188851184, 0.02996942805071622, 0.027944648835599783, 0.018420939810150447, 0.030881026281938635, 0.04424662191817478, 0.05034933977176647, 0.01275153054459495, 0.025955990308812165, 0.033824661172849484]
[0.011127060766212955, 0.016149897413068916, 0.013519521911848553, 0.014428820925449235, 0.06898222341895631, 0.056723010856930135, 0